In [ ]:
!pip install anthropic pillow pydub moviepy ffmpeg

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 9.6 MB/s eta 0:00:00
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=7a79171adeb56dbc12731775822c7dc2cd9b33f19c38753f77e1cf60f9e5bfe7
  Stored in directory: /root/.cache/pip/wheels/56/30/c5/576bdd729f3bc062d62a551be7fefd6ed2f761901568171e4e
Successfully built ffmpeg


In [ ]:
!pip install --upgrade pydub  

✅ Successfully unzipped content


In [ ]:
import zipfile
import os

def unzip_content():

    if os.path.exists('/content/chapters.zip'):
        with zipfile.ZipFile('/content/chapters.zip', 'r') as zip_ref:
            zip_ref.extractall('/content')
        print("✅ Successfully unzipped content")
    else:
        raise FileNotFoundError("content.zip not found in /content directory")


unzip_content()

✅ Successfully unzipped content


In [ ]:
import os
import json
import subprocess
import concurrent.futures
from moviepy.editor import *
from google.colab import drive


drive.mount('/content/drive')


audio_dir = "/content/audio"
timepoints_dir = "/content/timepoints"
chapters_dir = "/content/chapters"
output_dir = "/content/drive/My Drive/videos" 

os.makedirs(output_dir, exist_ok=True)

def load_timepoints(timepoints_path):
    """Load and parse timepoints from JSON file."""
    with open(timepoints_path, 'r') as f:
        timepoints = json.load(f)
    timepoints.sort(key=lambda x: x['timeSeconds'])
    return [point['timeSeconds'] for point in timepoints]

def create_chapter_video(chapter_num):
    """
    Creates a video with precise timing and exports using FFmpeg.
    """
    print(f"\n📖 Processing Chapter {chapter_num}")

    # Define paths
    audio_path = os.path.join(audio_dir, f"chapter{chapter_num}.mp3")
    timepoints_path = os.path.join(timepoints_dir, f"chapter{chapter_num}.json")
    img_folder = os.path.join(chapters_dir, f"chapter{chapter_num}")
    temp_video_path = f"/content/temp_chapter{chapter_num}.mp4"
    output_path = os.path.join(output_dir, f"chapter{chapter_num}.mp4")  # 🔥 Save to Google Drive


    if not os.path.exists(audio_path) or not os.path.exists(timepoints_path):
        print(f"Missing files for Chapter {chapter_num}")
        return


    audio_clip = AudioFileClip(audio_path)
    total_duration = audio_clip.duration


    transitions = load_timepoints(timepoints_path)


    images = sorted(
        [os.path.join(img_folder, f) for f in os.listdir(img_folder) if f[0].isdigit()],
        key=lambda x: int(os.path.basename(x).split('-')[0])
    )


    num_images = len(images)
    num_transitions = len(transitions)

    if num_images == 0:
        print(f"❌ No images found for Chapter {chapter_num}")
        return


    if num_images < num_transitions:
        transitions = transitions[:num_images]


    transitions.insert(0, 0) 
    transitions.append(total_duration)  

    clips = []
    for i in range(len(transitions) - 1):
        if i >= len(images):  
            break

        start_time = transitions[i]
        end_time = transitions[i + 1]
        duration = end_time - start_time

        if duration > 0:
            img_clip = ImageClip(images[i]).set_start(start_time).set_duration(duration)
            clips.append(img_clip)


    print(f"🎬 Creating video for Chapter {chapter_num}...")
    video = CompositeVideoClip(clips, size=clips[0].size)
    video = video.set_audio(audio_clip).set_duration(total_duration)


    video.write_videofile(
        temp_video_path,
        fps=10, codec="libx264", audio_codec="aac",
        preset="ultrafast", threads=os.cpu_count()
    )

    video.close()
    audio_clip.close()


    print(f"Converting Chapter {chapter_num} ")
    ffmpeg_cmd = [
        "ffmpeg", "-y", "-threads", str(os.cpu_count()),
        "-i", temp_video_path,
        "-i", audio_path,
        "-c:v", "libx264", "-preset", "ultrafast", "-crf", "23",
        "-c:a", "aac", "-b:a", "192k",
        "-shortest", output_path
    ]
    subprocess.run(ffmpeg_cmd, check=True)


    os.remove(temp_video_path)

    print(f"Chapter {chapter_num} saved to Google Drive in `videos` folder!")

def main():

    chapters = [
        int(filename[7:-4])
        for filename in sorted(os.listdir(audio_dir))
        if filename.startswith("chapter") and filename.endswith(".mp3")
    ]


    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = [executor.submit(create_chapter_video, ch) for ch in chapters]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error processing chapter: {str(e)}")

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

📖 Processing Chapter 1
🎬 Creating video for Chapter 1...
Moviepy - Building video /content/temp_chapter1.mp4.
MoviePy - Writing audio in temp_chapter1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/temp_chapter1.mp4



Moviepy - Done !
Moviepy - video ready /content/temp_chapter1.mp4
🚀 Converting Chapter 1 with FFmpeg...
✅ Chapter 1 saved to Google Drive in `videos` folder!
